# Lecture 04 – part II
                                       
## Introduction to Data-munging
                                       
   - Add new manipulated variables     
   - Separate char to new variables    
   - Convert variables to numeric or factor               
   - Some string manipulations         
   - Rename variables                  
   - Filter out different observations 
     - conditional selection           
     - tabulate frequency of a var     
     - missing values                  
     - replace values                  
     - duplicates                      
   - Using pipes                
   - Sorting data                      
                                       
#### Case-study 

- Based on: Chapter 02, A:              
    - Finding a good deal among hotels: data preparation              
                                      
#### Dataset:                              
- hotels-europe 
---

In [9]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

Import raw data

In [10]:
raw_df = pd.read_csv("https://osf.io/yzntm/download")

Have a look at the data

In [11]:
raw_df.head()

,addresscountryname,city_actual,rating_reviewcount,center1distance,center1label,center2distance,center2label,neighbourhood,price,price_night,...,accommodationtype,guestreviewsrating,scarce_room,hotel_id,offer,offer_cat,year,month,weekend,holiday
0,Netherlands,Amsterdam,1030.0,3.1 miles,City centre,3.6 miles,Montelbaanstoren,Amsterdam,172,price for 1 night,...,_ACCOM_TYPE@Hotel,4.3 /5,0,1.0,0,0% no offer,2017,11,1,0
1,Netherlands,Amsterdam,1030.0,3.1 miles,City centre,3.6 miles,Montelbaanstoren,Amsterdam,122,price for 1 night,...,_ACCOM_TYPE@Hotel,4.3 /5,0,1.0,1,15-50% offer,2018,1,1,0
2,Netherlands,Amsterdam,1030.0,3.1 miles,City centre,3.6 miles,Montelbaanstoren,Amsterdam,122,price for 1 night,...,_ACCOM_TYPE@Hotel,4.3 /5,0,1.0,1,15-50% offer,2017,12,0,1
3,Netherlands,Amsterdam,1030.0,3.1 miles,City centre,3.6 miles,Montelbaanstoren,Amsterdam,552,price for 4 nights,...,_ACCOM_TYPE@Hotel,4.3 /5,0,1.0,1,1-15% offer,2017,12,0,1
4,Netherlands,Amsterdam,1030.0,3.1 miles,City centre,3.6 miles,Montelbaanstoren,Amsterdam,122,price for 1 night,...,_ACCOM_TYPE@Hotel,4.3 /5,0,1.0,1,15-50% offer,2018,2,1,0


### 1) Adding a new variable

Let us add nnights variable, which shows the number of nights spent in the hotel\
As the data was collected in such way, it is 1 for each observations.

You can assign a new variable using the following syntax

In [20]:
raw_df["nnights"] = 1

Or use the `assign` function of pandas and also create a new variable, lets call it `df`

In [21]:
df = raw_df.assign(nnights = 1)

Lets remove the raw data

In [22]:
del raw_df

### 2) Data cleaning - selecting a variable

You can select variables with `[]` brackets after the DataFrame object

In [23]:
df[["accommodationtype" , "price"]]

,accommodationtype,price
0,_ACCOM_TYPE@Hotel,172
1,_ACCOM_TYPE@Hotel,122
2,_ACCOM_TYPE@Hotel,122
3,_ACCOM_TYPE@Hotel,552
4,_ACCOM_TYPE@Hotel,122
...,...,...
149961,_ACCOM_TYPE@Hotel,119
149962,_ACCOM_TYPE@Hotel,109
149963,_ACCOM_TYPE@Hotel,109
149964,_ACCOM_TYPE@Hotel,446


Or use the `filter` function

In [24]:
df.filter(["accommodationtype" , "price"])

,accommodationtype,price
0,_ACCOM_TYPE@Hotel,172
1,_ACCOM_TYPE@Hotel,122
2,_ACCOM_TYPE@Hotel,122
3,_ACCOM_TYPE@Hotel,552
4,_ACCOM_TYPE@Hotel,122
...,...,...
149961,_ACCOM_TYPE@Hotel,119
149962,_ACCOM_TYPE@Hotel,109
149963,_ACCOM_TYPE@Hotel,109
149964,_ACCOM_TYPE@Hotel,446


You can also select columns with `filter` using Regular Expressions aka regex

In [25]:
df.filter(regex="rating")

,rating_reviewcount,starrating,rating2_ta,rating2_ta_reviewcount,guestreviewsrating
0,1030.0,4.0,4.0,1115.0,4.3 /5
1,1030.0,4.0,4.0,1115.0,4.3 /5
2,1030.0,4.0,4.0,1115.0,4.3 /5
3,1030.0,4.0,4.0,1115.0,4.3 /5
4,1030.0,4.0,4.0,1115.0,4.3 /5
...,...,...,...,...,...
149961,48.0,4.0,4.5,86.0,4.5 /5
149962,48.0,4.0,4.5,86.0,4.5 /5
149963,48.0,4.0,4.5,86.0,4.5 /5
149964,48.0,4.0,4.5,86.0,4.5 /5


### 3) separating character vector with a unique sign 

 Check accomotationtype: it is a character with a clear separator "@"

 To clean accommodationtype column: separate the characters at @ 
   and create two new variables: "garbage" and "acc_type".
 garbage will contain all characters before @ sign and acc_type will take everything after!

In [26]:
df["accommodationtype"]

0         _ACCOM_TYPE@Hotel
1         _ACCOM_TYPE@Hotel
2         _ACCOM_TYPE@Hotel
3         _ACCOM_TYPE@Hotel
4         _ACCOM_TYPE@Hotel
                ...        
149961    _ACCOM_TYPE@Hotel
149962    _ACCOM_TYPE@Hotel
149963    _ACCOM_TYPE@Hotel
149964    _ACCOM_TYPE@Hotel
149965    _ACCOM_TYPE@Hotel
Name: accommodationtype, Length: 149966, dtype: object

You can easily split and strip character columns wiht pandas with attaching the `str` function after column name

In [27]:
df["accommodationtype"].str.split("@")

0         [_ACCOM_TYPE, Hotel]
1         [_ACCOM_TYPE, Hotel]
2         [_ACCOM_TYPE, Hotel]
3         [_ACCOM_TYPE, Hotel]
4         [_ACCOM_TYPE, Hotel]
                  ...         
149961    [_ACCOM_TYPE, Hotel]
149962    [_ACCOM_TYPE, Hotel]
149963    [_ACCOM_TYPE, Hotel]
149964    [_ACCOM_TYPE, Hotel]
149965    [_ACCOM_TYPE, Hotel]
Name: accommodationtype, Length: 149966, dtype: object

This results in a Series of lists, each list contains the splitted parts of the original string. You can access the with the following syntax

In [28]:
df["accommodationtype"].str.split("@").str[0]

0         _ACCOM_TYPE
1         _ACCOM_TYPE
2         _ACCOM_TYPE
3         _ACCOM_TYPE
4         _ACCOM_TYPE
             ...     
149961    _ACCOM_TYPE
149962    _ACCOM_TYPE
149963    _ACCOM_TYPE
149964    _ACCOM_TYPE
149965    _ACCOM_TYPE
Name: accommodationtype, Length: 149966, dtype: object

In [29]:
df["accommodationtype"].str.split("@").str[1].str.strip()

0         Hotel
1         Hotel
2         Hotel
3         Hotel
4         Hotel
          ...  
149961    Hotel
149962    Hotel
149963    Hotel
149964    Hotel
149965    Hotel
Name: accommodationtype, Length: 149966, dtype: object

The second value in the lists contains the type of information about the accomodation type, lets create a new variable out of it.

In [30]:
df["acc_type"] = df["accommodationtype"].str.split("@").str[1].str.strip()
df["acc_type"].value_counts()

Hotel                  102234
Apartment               17113
Guest House             10080
Bed and breakfast        6395
Hostel                   6232
Apart-hotel              3693
Vacation home Condo      1576
Pension                  1085
Inn                      1020
Country House              89
Palace                     68
                           65
Villa                      64
Resort                     60
Caravan Park               60
House boat                 31
Pousada                    28
Motel                      25
Cabin / Lodge              14
Cottage                    13
Chalet                     11
Castle                      5
All inclusive               5
Name: acc_type, dtype: int64

Its a good idea to use the `strip` function when splitting string variables as it removes any unnecessary white space

### Task - creating a numeric vector w simple separation

  1) Correct the `guestreviewsrating` variable into simple numeric variable\
  2) Check with `.dtypes` method\
  3) Convert the variable into a numeric variable


In [33]:
df["guestreviewsrating"].value_counts(dropna=False)

4 /5      23160
3.7 /5    15031
4.5 /5    14952
4.3 /5    11268
4.1 /5    11190
3.5 /5    11145
NaN       10587
4.4 /5     9079
3.9 /5     8853
3.2 /5     6112
3 /5       4838
4.6 /5     4417
3.4 /5     3727
4.8 /5     3095
5 /5       2646
2.7 /5     2337
2.5 /5     1557
2.9 /5     1213
4.9 /5     1157
2 /5        976
2.2 /5      976
1 /5        679
2.4 /5      472
1.7 /5      143
1.8 /5      112
1.5 /5       91
1.6 /5       54
1.2 /5       48
1.3 /5       41
1.1 /5       10
Name: guestreviewsrating, dtype: int64

In [34]:
df["ratings"] = df["guestreviewsrating"].str.split('/').str[0].str.strip()
df["ratings"].dtypes

dtype('O')

In [35]:
df["ratings"] = df["ratings"].astype(float)

In [36]:
df["ratings"].dtypes

dtype('float64')

In [37]:
df["ratings"].value_counts(dropna=False)

4.0    23160
3.7    15031
4.5    14952
4.3    11268
4.1    11190
3.5    11145
NaN    10587
4.4     9079
3.9     8853
3.2     6112
3.0     4838
4.6     4417
3.4     3727
4.8     3095
5.0     2646
2.7     2337
2.5     1557
2.9     1213
4.9     1157
2.0      976
2.2      976
1.0      679
2.4      472
1.7      143
1.8      112
1.5       91
1.6       54
1.2       48
1.3       41
1.1       10
Name: ratings, dtype: int64

### Task

Create numerical variables from the two distance measures:

In [ ]:
df[["center1distance", "center2distance"]]

We have two numeric values than the format of the distance "miles"

In [ ]:
df["distance"] = df["center1distance"].str.strip().str.split().str[0].astype(float)
df["distance_alter"] = df["center2distance"].str.strip().str.split().str[0].astype(float)

Check results

In [ ]:
df["distance"].hist(bins=20)

### Rename variables

Using the `rename` function, you have to specify in a dictionary that which columns you want to rename and their new names

In [ ]:
df.filter(regex="rating")

In [ ]:
df = df.rename(columns={"rating_reviewcount": "rating_count", "rating2_ta": "ratingta"})

In [ ]:
df.filter(regex="rating")

### Task:
   
also rename the following variables as follows:
     
    rating2_ta_reviewcount: ratingta_count
    addresscountryname: "country
    starrating: stars
    s_city": city

In [ ]:
df = df.rename(columns={
    "rating2_ta_reviewcount": "ratingta_count",
    "addresscountryname": "country",
    "starrating": "stars",
    "s_city": "city",
})

### Filtering observations

Use the `loc` function

In [ ]:
df.shape

let us have only hotels

In [ ]:
df.loc[df["acc_type"] == "Hotel"]

Filtering: find missing values\
look at one of our key variable: ratings\
we can calculate the frequencies of the ratings


In [ ]:
df["ratings"].isnull().sum()

What can we do with the NA values?\
First check them with 'loc'


In [ ]:
df.loc[df["ratings"].isnull()]

if reasonable we can drop them, but there needs to be good reason for that!


In [ ]:
df = df.loc[df["ratings"].notnull()]

Or alternatively ose `dropna`

In [ ]:
df = df.dropna(subset=["ratings"])

In [ ]:
df

### Correcting wrongly documented observations:
 In case of `stars` there are only values from 0-5

In [ ]:
df["stars"].value_counts(dropna=False)

What does 0 star means? It is missing, but recorded as 0...\
We need to set these values to `None`: re-define the stars variable:

In [ ]:
df["stars"] = np.where(df["stars"] == 0, None, df["stars"])

In [ ]:
df["stars"].value_counts(dropna=False)

### Duplicates:

1.) exact match for each values for a given observations\
 
Count the number of duplicates

In [ ]:
df.duplicated().sum()

Remove them

In [ ]:
df = df.drop_duplicates()

2.) Remove duplicates to specific variables, that are important to us

To make sense, let us take this into two steps: 

    a) select certain variables:


In [ ]:
important_variables = [
    "country",
    "hotel_id",
    "distance",
    "stars",
    "ratings",
    "price",
    "year",
    "month",
    "weekend",
    "holiday",
]

 Now we can filter out only duplicates in these variables:

In [ ]:
df = df.drop_duplicates(subset=important_variables)

### Task: Get specific data used in DA1 course:
       1) Get hotels only from Vienna
       2) Filter out the following observations:
           - in date: 2017, November and 0s week (multiple conditions)
           - with Hotel types which has stars between 3 and 4
           - and drop observations which has price more than 1000 EUR.


In [ ]:
hotel_vienna = df.loc[df["city"] == "Vienna"]
hotel_vienna = hotel_vienna.loc[
    (hotel_vienna["year"] == 2017)
    & (hotel_vienna["month"] == 11)
    & (hotel_vienna["weekend"] == 0)
]  # use | and & for bitwise operations
hotel_vienna = hotel_vienna.loc[hotel_vienna["acc_type"] == "Hotel"]
hotel_vienna = hotel_vienna.loc[
    (hotel_vienna["stars"] >= 3) & (hotel_vienna["stars"] <= 4)
]
hotel_vienna = hotel_vienna.loc[hotel_vienna["price"] < 1000]

In [ ]:
hotel_vienna

### Pipes with pandas

You can also do the above procedure in a pipe, which allows to make multiple sequential manipulations with one command:

In [ ]:
(
    df.loc[df["city"] == "Vienna"]
    .loc[(df["year"] == 2017) & (df["month"] == 11) & (df["weekend"] == 0)]
    .loc[df["acc_type"] == "Hotel"]
    .loc[(df["stars"] >= 3) & (df["stars"] <= 4)]
    .loc[df["price"] < 1000]
)

with pipeing, you can lace several different functions in one pipe, eg.

In [ ]:
(
    df.loc[(df["year"] == 2017) & (df["month"] == 11) & (df["weekend"] == 0)]
    .rename(columns={"city": "town"})
    .filter(["hotel_id", "country", "town", "price", "acc_type"])
    .groupby("town")
    .agg(n_hotels=("hotel_id", "nunique"), mean_price=("price", "mean"))
    .sort_values(by=["mean_price"], ascending=False)
    .head(10)
)

Make data table more "pretty"

Can arrange the values in increasing order

In [ ]:
hotel_vienna.sort_values(by=["price"], ascending=True)

Or decreasing order

In [ ]:
hotel_vienna.sort_values(by=["price"], ascending=False)

### Task: writing out csv as clean data

In [ ]:
hotel_vienna.to_csv("hotel_vienna_restricted.csv", index=False)